In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

from numpy import linalg as LA
from time import time
from pprint import pprint
from copy import deepcopy as copy


%matplotlib inline

## Read data files

**Note** We are keeping only the 50 states. DC not included. Self loops not included

In [2]:
def read_US_states(fname):
    states_abb_dict = {}
    states_abb_rev_dict = {}
    states_abb_ord_list = []
    with open(fname, "r") as f:
        for line in f:
            line = line.strip().split(",")
            name = line[0]
            abbr = line[1]
            states_abb_dict[name] = (abbr, len(states_abb_ord_list))
            states_abb_rev_dict[abbr] = name
            states_abb_ord_list.append(abbr)
    
    return states_abb_dict, states_abb_rev_dict, states_abb_ord_list

def read_travel_network(fname, states_abb_dict, states_abb_rev_dict, normalization=1):
    num_states = 50

    adjacency_list = {}
    A = np.zeros((num_states, num_states))

    for abb in states_abb_rev_dict:
        adjacency_list[abb] = []

    with open(fname, "r") as f:
        for idx, line in enumerate(f):
            if idx == 0:
                continue
            line = line.strip().split(",")
            orig = line[0]
            dest = line[1]
            weight = float(line[2])*10 / normalization
            if orig == dest:
                continue
            try:
                orig_abb, orig_idx = states_abb_dict[orig]
                dest_abb, dest_idx = states_abb_dict[dest]
                adjacency_list[orig_abb].append((dest_abb, weight))
                A[orig_idx][dest_idx] = weight
            except KeyError:
                pass
    
    return adjacency_list, A

def read_deaths_data(fname):
    df = pd.read_csv(fname)
    sum_df_d = df.groupby(['State', 'Year', 'Quarter'])[['DeathsFromPneumoniaAndInfluenza']].sum().T.to_dict()
    deaths_dict = {}
    for key, deaths in sum_df_d.items():
        state, year, quarter = key
        if state == "DC":
            continue
        if state not in deaths_dict:
            deaths_dict[state] = {}
        if year not in deaths_dict[state]:
            deaths_dict[state][year] = {}
        deaths_dict[state][year][quarter] = deaths["DeathsFromPneumoniaAndInfluenza"]
    return deaths_dict

def read_population_dict(fname):
    population_dict = {}
    with open(fname, "r") as f:
        for idx, line in enumerate(f):
            if idx == 0:
                continue
            line = line.strip().split(",")
            state = line[1]
            population_dict[state] = {}
            year = 2009
            for idx, pop in enumerate(line[2:]):
                population_dict[state][year + idx] = int(pop)
    return population_dict

## Simulation


1. Randomly infect people in each state with probability `p_inf`
    - Calculate fraction of infected people
2. Air travel the people
    - Fraction of infected air travelers is the same as origin state
    - Each person has a chance to infect every other person in the new state with probability `p_transfer`
3. In each state, either recover, kill, or remain infected the people there with probability `p_rec, p_die, p_stay`
4. Increment, or decrement, population between years (not quarters)

Totally, there are 4 parameters: `p_inf`, `p_transfer`, `p_rec`, `p_die`

Note `p_stay = 1 - p_rec - p_die`

Can make into a 3 parameter model by setting `p_stay = 0` i.e., between quarters, people either heal completely, or die.

### Goal: Find the best parameters that represent the number of deaths observed

In [3]:
class State:
    def __init__(self, pop):
        self.num_total = pop
        self.num_infected = 0
        self.num_deceased = 0
    
    def infected_fraction(self):
        return self.num_infected / self.num_total

### Kernels for simulation

In [66]:
def travel_and_infect_kernel(A, states, p_transfer, verbose=False):
    N = A.shape[0]
    new_states = copy(states)
    
    infected_fraction = np.zeros((N,))
    for i in range(N):
        infected_fraction[i] = states[i].infected_fraction()
    
    touch_fraction = 0.5
    
    for dest_id in range(N):
        
        # Let X be number of successful infections.
        # X is Binomial(n, p_transfer) where n is number of travelers.
        # We want X >= 1 for each person at destination i.e.,
        # at least one successful infection for each person at destination.
        # So, calculate p' = P(X >= 1) = 1 - P(X = 0) = 1 - (1-p_transfer)^n.
        # p' is the probability a person at destination gets infected.
        # This reduces problem to calculate a new r.v. Y.
        # Y is Binomial(m, p'), where m is the uninfected population of destination
        
        num_immigrants = A[:, i]
        num_infected_immigrants = np.matmul(num_immigrants, infected_fraction.T)
        # in expectation:
        # num_infected_immigrants = np.sum(A[:, i]) * p_inf
        ccdf = 1 - (1 - p_transfer)**num_infected_immigrants
        dest = states[dest_id]
        if dest.num_total - dest.num_infected > 0:
            if np.isnan(ccdf):
                ccdf = 1
            new_states[dest_id].num_infected += np.random.binomial(
                touch_fraction*(dest.num_total - dest.num_infected), ccdf)
#             # in expectation:
#             new_states[dest_id].num_infected += (dest.num_total - dest.num_infected) * ccdf
        
    return new_states

def recover_kernel(states, SIR_params):
    SIR_params = np.abs(SIR_params) / np.sum(np.abs(SIR_params))
    for i, state in enumerate(states):
        x = np.random.multinomial(states[i].num_infected, SIR_params)
        recovered = x[0]
        dead = x[2]
        state.num_total = max(state.num_total - dead, 0)
        states[i].num_infected -= (dead + recovered)
        states[i].num_deceased += dead
    return states

def local_infection_kernel(states, p_transfer):
    for i, state in enumerate(states):
        states[i].num_infected = np.random.binomial(state.num_total - state.num_infected, p_transfer)
    return states

def remove_deceased_kernel(states):
    num_deceased = []
    for i, state in enumerate(states):
        num_deceased.append(state.num_deceased)
        state.num_deceased = 0
        states[i] = state
    return states, num_deceased
    
def inject_population_kernel(states, new_population):
    # new_population is somehow ordered
    for i, state in enumerate(states):
        current_pop = state.num_total
        nextgen_pop = new_population[i]
        extra_peeps = nextgen_pop - current_pop
        if extra_peeps > 0:
            state.num_total += extra_peeps
        elif extra_peeps < 0:
            # population actually decreased... remove infected and susceptible at same rate
            extra_peeps = -extra_peeps
            if extra_peeps // 2 >= state.num_infected:
                state.num_total = state.num_total - (extra_peeps - state.num_infected)
                state.num_infected = 0
            else:
                state.num_infected -= (extra_peeps - extra_peeps//2)
                state.num_total -= extra_peeps
        states[i] = state
    return states

def random_infection_kernel(states, p_inf):
    for i, state in enumerate(states):
        if state.num_total < state.num_infected:
            continue
        infected = np.random.binomial(state.num_total - state.num_infected, p_inf)
        state.num_infected += infected
        states[i] = state
    return states

### Real data

In [6]:
data_dir = "../Data/Clean/"
deaths_fname = "deaths_NCHS_processed.csv"
population_fname = "population.csv"

states_abb_dict, states_abb_rev_dict, stats_abb_ord_list = read_US_states(data_dir + "states_abb.csv")
deaths_dict = read_deaths_data(data_dir + deaths_fname)
population_dict = read_population_dict(data_dir + population_fname)

adj_list = {}
A = {}
A[2009] = {}
A[2009][4] = read_travel_network(data_dir + "2009_Q4.csv", states_abb_dict, states_abb_rev_dict)[1]
for year in range(2010, 2019):
    A[year] = {}
    for quarter in range(1, 5):
        network_fname = str(year) + "_Q" + str(quarter) + ".csv"
        A[year][quarter] = read_travel_network(data_dir + network_fname, states_abb_dict, states_abb_rev_dict)[1]
A[2019] = {}
A[2019][1] = read_travel_network(data_dir + "2019_Q1.csv", states_abb_dict, states_abb_rev_dict)[1]

all_population = {}
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
for year in years:
    all_population[year] = []


for state,data in population_dict.items():
    for year, pop in data.items():
        all_population[year].append(pop)
        if year == 2018:
            all_population[2019].append(pop)
        
A_dict = A

In [7]:
deaths_matrix = {-1: [], 1: [], 2: [], 3: [], 4:[]}
for state, yearly_deaths_dict in deaths_dict.items():
    deaths_vector = {}
    for q in deaths_matrix:
        deaths_vector[q] = []
    for year, quarterly_deaths_dict in yearly_deaths_dict.items():
#         if year > 2012:
#             continue
        for quarter, deaths_count in quarterly_deaths_dict.items():
            deaths_vector[-1].append(deaths_count)
            deaths_vector[quarter].append(deaths_count)
    for q in deaths_matrix:
        deaths_matrix[q].append(deaths_vector[q])
        

for q in deaths_matrix:
    deaths_matrix[q] = np.array(deaths_matrix[q])
    print(deaths_matrix[q].shape)

(50, 38)
(50, 10)
(50, 9)
(50, 9)
(50, 10)


In [67]:
def run_full_simulation(x, A_dict, all_population, error=False, sim_quarter=-1):
    p_inf = x[0]
    p_transfer = x[1]
    if p_inf < 0 or p_inf > 1 or p_transfer < 0 or p_transfer > 1:
        return 10
    p_rec = x[2]
    p_die = 7.540044190323758e-05
#     p_die = x[3]
    p_stay = 1 - p_rec - p_die
    SIR = [p_rec, p_stay, p_die]
#     SIR = np.abs(SIR / np.abs(np.sum(SIR)))
    states = []
    year_0 = 2009
    for state_id in range(50):
        state = State(all_population[year_0][state_id])
        state.num_infected += np.random.binomial(state.num_total, p_inf)
        states.append(state)

    # Propagate infection
    num_deceased = {}

    # For each year:
    for year in years:
        num_deceased[year] = {}
        # For each quarter:
        for quarter in range(1, 5):
            if year == 2009 and quarter != 4:
                continue
            if year == 2019 and quarter != 1:
                continue
#             if year > 2012:
#                 continue
            if sim_quarter != -1 and quarter != sim_quarter:
                continue
            # 1. travel and infect
            # 2. Recover
            # 3. Remove deceased and store it
            # 4. Randomly infect
            states = travel_and_infect_kernel(A_dict[year][quarter], states, p_transfer)
            states = recover_kernel(states, SIR)
            
            for i in range(4):
                states = local_infection_kernel(states, p_transfer)
                states = recover_kernel(states, SIR)
            
            states, dead_peeps = remove_deceased_kernel(states)
            num_deceased[year][quarter] = dead_peeps
            states = random_infection_kernel(states, p_inf)

        # Inject population
        try:
            pop_vec = inject_population_kernel(states, all_population[year+1])
        except KeyError:
            # We are at the end of our data
            pass
    
    sim_deaths_matrix = []

    for year, quarterly_deaths in num_deceased.items():
        for quarter, deaths_vector in quarterly_deaths.items():
            sim_deaths_matrix.append(deaths_vector)

    sim_deaths_matrix = np.array(sim_deaths_matrix).T
    
    if error:
        return LA.norm(deaths_matrix[sim_quarter] - sim_deaths_matrix, ord='fro') / LA.norm(deaths_matrix[sim_quarter], ord='fro')
    else:
        return sim_deaths_matrix

In [148]:
# Fix parameters of the model
P_inf = np.linspace(0.05, 0.2, num=100)
P_transfer = np.linspace(0, 1, num=100)
P_rec = np.linspace(0, 1, num=100)
p_die = 7.540044190323758e-05

x0 = np.array([P_inf[23], P_transfer[10], P_rec[33]])
run_full_simulation(x0, A_dict, all_population, error=True)

2.0437698030389284

### Random Walks?

In [69]:
def perturbe(x, mu=0, sigma=1e-4):
    return x + np.random.normal(mu, sigma, size=x.shape)

def isvalid(x):
    return (x <= 1).all() and (x >= 0).all()

error_tol = 0.5
quarter = 2

# x_best = np.array([1.50141914e-03, 5.08739999e-04, 9.85804602e-01])
# x_best = np.array([2.44585049e-02, 5.19930292e-05, 9.11488483e-01])
# x_best = np.array([2.47333730e-02, 8.97588162e-05, 9.79676153e-01])
# x_best = np.array([2.45480162e-02, 3.68910748e-04, 9.79240520e-01, 2.88207661e-05])
x_best = np.array([0.1, 0.1, 0.99])
# x_best = np.random.uniform(0, 1, size=(3,))
print(x_best)
error = run_full_simulation(x_best, A_dict, all_population, True, sim_quarter=quarter)
print("0", error)
error_best = error

ctr = 0
invalid_ctr = 0

while error > error_tol:
    x_i = perturbe(x_best, mu=0, sigma=1e-1)
#     x_i = x_best + np.random.normal(0, 1e-3, size=x_best.shape)
    if not isvalid(x_i):
        invalid_ctr += 1
        if invalid_ctr % 10 == 0:
            print("Invalid: ", invalid_ctr)
        continue
    error = run_full_simulation(x_i, A_dict, all_population, True, sim_quarter=quarter)
    ctr += 1
    if error < error_best:
        x_best = copy(x_i)
        error_best = error
        print(ctr, error)
    elif ctr % 10 == 0:
        print("Counter: ", ctr)

[0.1  0.1  0.99]
0 0.6326334552051824
Invalid:  10
1 0.6164469085428642
2 0.6159811906702415
6 0.560996401486404
9 0.560081712511025
Counter:  10
Counter:  20
Counter:  30
Counter:  40
Counter:  50
Counter:  60
Counter:  70
Counter:  80
Invalid:  20
Counter:  90
Counter:  100
Counter:  110


KeyboardInterrupt: 

In [54]:
print(x_best)
sim_deaths = run_full_simulation(np.array([0.1, 0.1, 0.99]), A_dict, all_population, sim_quarter=quarter)

[0.14619921 0.15400028 0.84293936]


In [55]:
fraction = np.abs(deaths_matrix[quarter] - sim_deaths)# / deaths_matrix[-1]
np.mean(fraction)

554.1121052631579

In [56]:
diff = (deaths_matrix[quarter] - sim_deaths).flatten()
print(np.mean(diff))
print(np.std(diff))

283.7763157894737
776.5110071183153


In [57]:
sim_deaths

array([[508, 490, 535, ..., 545, 504, 521],
       [ 70,  95,  72, ...,  61,  83,  85],
       [678, 684, 669, ..., 741, 678, 726],
       ...,
       [214, 203, 202, ..., 206, 191, 199],
       [576, 583, 617, ..., 627, 659, 595],
       [ 56,  46,  48, ...,  68,  56,  66]])

In [58]:
deaths_matrix[quarter]

array([[  59.,   56.,   61., ...,   50.,   52.,   66.],
       [ 893., 1051.,  817., ...,  643.,  733.,  783.],
       [ 667.,  710.,  542., ...,  455.,  528.,  606.],
       ...,
       [ 946.,  925.,  760., ...,  612.,  653.,  739.],
       [ 527.,  578.,  449., ...,  340.,  376.,  289.],
       [ 108.,  111.,  103., ...,   74.,   86.,   93.]])

need beautiful plots!